In [25]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [51]:
base = pd.read_csv('test.csv')

In [52]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classeb

In [53]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
x = scaler_x.fit_transform(x)

In [54]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.3)

In [55]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape

(150,)

In [56]:
y = y.values.reshape(-1,1)
y.shape

(150, 1)

In [57]:
x.shape

(150, 1000)

In [58]:
y = onehot.fit_transform(y).toarray()
y.shape

(150, 3)

In [59]:
neuronios_entrada = x.shape[1]
neuronios_entrada

1000

In [60]:
neuronios_oculta1 = int((neuronios_entrada + y.shape[1])/2)
neuronios_oculta1

501

In [61]:
neuronios_oculta2 = neuronios_oculta1
neuronios_oculta3 = neuronios_oculta1
neuronios_saida = y.shape[1]
neuronios_saida

3

In [38]:
# 1000 -> 501 -> 501 -> 501 -> 3

In [62]:
w = {'oculta1': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta1])),
     'oculta2': tf.Variable(tf.random_normal([neuronios_oculta1, neuronios_oculta2])),
     'oculta3': tf.Variable(tf.random_normal([neuronios_oculta2, neuronios_oculta3])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta3, neuronios_saida]))}
w

{'oculta1': <tf.Variable 'Variable_19:0' shape=(1000, 501) dtype=float32_ref>,
 'oculta2': <tf.Variable 'Variable_20:0' shape=(501, 501) dtype=float32_ref>,
 'oculta3': <tf.Variable 'Variable_21:0' shape=(501, 501) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_22:0' shape=(501, 3) dtype=float32_ref>}

In [63]:
b = {'oculta1': tf.Variable(tf.random_normal([neuronios_oculta1])),
     'oculta2': tf.Variable(tf.random_normal([neuronios_oculta2])),
     'oculta3': tf.Variable(tf.random_normal([neuronios_oculta3])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}
b

{'oculta1': <tf.Variable 'Variable_23:0' shape=(501,) dtype=float32_ref>,
 'oculta2': <tf.Variable 'Variable_24:0' shape=(501,) dtype=float32_ref>,
 'oculta3': <tf.Variable 'Variable_25:0' shape=(501,) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_26:0' shape=(3,) dtype=float32_ref>}

In [64]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [65]:
def mlp(x, w, bias):
    camada_oculta1 = tf.nn.relu(tf.add(tf.matmul(x, w['oculta1']), b['oculta1']))
    camada_oculta2 = tf.nn.relu(tf.add(tf.matmul(camada_oculta1, w['oculta2']), b['oculta2']))
    camada_oculta3 = tf.nn.relu(tf.add(tf.matmul(camada_oculta2, w['oculta3']), b['oculta3']))
    camada_saida = tf.add(tf.matmul(camada_oculta3, w['saida']), b['saida'])
    return camada_saida

In [66]:
modelo = mlp(xph, w, b)
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(erro)

In [67]:
batch_size = 8
batch_total = int(len(x_treinamento) / batch_size)
x_batches = np.array_split(x_treinamento, batch_total)
x_batches

[array([[ 0.9860084 ,  0.99302578,  1.0000573 , ..., -1.04822626,
         -1.04101563,  0.9860084 ],
        [ 1.33188484,  1.34074945,  1.34963723, ..., -1.38861471,
         -1.37962925,  1.33188484],
        [ 0.86045526,  0.85567493,  0.85043709, ...,  1.5719139 ,
          1.55547355,  0.86045526],
        ...,
        [-0.29996019, -0.3126766 , -0.3257246 , ...,  0.43467606,
          0.42162587, -0.29996019],
        [ 0.81307018,  0.81916395,  0.82526733, ..., -0.87803203,
         -0.87170883,  0.81307018],
        [ 0.12131731,  0.1237166 ,  0.12610747, ..., -0.19725511,
         -0.1944816 ,  0.12131731]]),
 array([[ 1.17502987,  1.17853636,  1.18183886, ...,  1.8697538 ,
          1.85734759,  1.17502987],
        [-0.22646144, -0.22661499, -0.22679348, ...,  0.48267083,
          0.48156048, -0.22646144],
        [-0.05162091, -0.05014523, -0.0486825 , ..., -0.02706088,
         -0.02517479, -0.05162091],
        ...,
        [ 0.9860084 ,  0.97077147,  0.95531107, ..., -

In [68]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epocas in range (500):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 100 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])

ValueError: Cannot feed value of shape (9,) for Tensor 'Placeholder_5:0', which has shape '(?, 3)'